# Planning Graph Search

In [60]:
import copy
import numpy

In [61]:
initial_state = []

In [62]:
class In:
    def __init__(self, obj, place, negation = False):
        self.object = obj
        self.place = place
        self.negation = negation
        
    def __hash__(self):
        return hash((self.object, self.place, self.negation))
        
    def __eq__(self, other):
        return isinstance(other, In) and self.object == other.object and self.place == other.place and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return In(self.object, self.place, negation = False)
        else:
            return In(self.object, self.place, negation = True)
        
    
       
    def __str__(self):
        if self.negation:
            return f'~In({self.object}, {self.place})'
        else:   
            return f'In({self.object}, {self.place})'
        

class Holding:
    def __init__(self, obj, negation = False):
        self.object = obj
        self.negation = negation
        
    def __hash__(self):
        return hash((self.object, self.negation))
        
    def __eq__(self, other):
        return isinstance(other, Holding) and self.object == other.object and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return Holding(self.object, negation = False)
        else:
            return Holding(self.object, negation = True)
    
    def __str__(self):
        if self.negation:
            return f'~Holding({self.object})'
        else:
            return f'Holding({self.object})'
        
class Locked:
    def __init__(self, obj, negation = False):
        self.object = obj
        self.negation = negation
        
    def __hash__(self):
        return hash((self.object, self.negation))
        
    def __eq__(self, other):
        return isinstance(other, Locked) and self.object == other.object and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
             return Locked(self.object, negation = False)
        else:
            return Locked(self.object, negation = True)
        
    def __str__(self):
        if self.negation:
            return f'~Locked({self.object})'
        else:
            return f'Locked({self.object})'
    
        


In [63]:
initial_state.append(In('Robot', 'R2'))
initial_state.append(In('Key', 'R2'))
initial_state.append(Locked('Door', negation = True))

In [64]:
print(' ^ '.join([str(x) for x in initial_state]))

In(Robot, R2) ^ In(Key, R2) ^ ~Locked(Door)


In [65]:
class GraspKey:
    def __init__(self):
        self.preconditions = [In('Robot', 'R2'), In('Key', 'R2')]
        self.implications = [~In('Robot', 'R1'), ~In('Key', 'R1')]
        self.delete = []
        self.effect = [Holding('Key')]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Grasp Key'
    
class LockDoor:
    def __init__(self):
        self.preconditions = [Holding('Key'), ~Locked('Door')]
        self.implications = []
        self.delete = [~Locked('Door')]
        self.effect = [Locked('Door')]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Lock Door'
    
class PutKeyIntoBox:
    def __init__(self):
        self.preconditions = [In('Robot', 'R1'),Holding('Key'), In('Key', 'R1')]
        self.implications = [~In('Robot', 'R2'), ~In('Key', 'R2')]
        self.delete = [Holding('Key'), In('Key', 'R1')]
        self.effect = [In('Key', 'Box')]
    def action(self, state):
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Put the key into the box'
    
class Move:
    def __init__(self, from_, to_):
        self.ubication = from_
        self.destiny = to_
        self.preconditions = [~Locked('Door'), In('Robot', self.ubication)]
        self.implications = [~In('Robot', self.destiny)]
        self.delete = [In('Robot', self.ubication)]
        self.effect = [In('Robot', self.destiny)]
    def action(self, state):
        if Holding('Key') in state:
            self.delete.append(In('Key', self.ubication))
            self.effect.append(In('Key', self.destiny))
        state = copy.deepcopy(state)
        if all(precondition in state for precondition in self.preconditions):
            for element in self.delete:
                if element in state:
                    state.remove(element)
            for element in self.effect:
                state.append(element)
        return list(set(state))
    def __str__(self):
        return f'Move from {self.ubication} to {self.destiny}'
        
    

In [66]:
def mutex(action1, action2):
    if any(elem in action1.delete for elem in action2.effect):
        return True
    if any(elem in action1.effect for elem in action2.delete):
        return True
    if any(elem in action1.delete for elem in action2.preconditions):
        return True
    if any(elem in action1.preconditions for elem in action2.delete):
        return True
    if any(~elem in action1.implications for elem in action2.preconditions):
        return True
    if any(~elem in action1.preconditions for elem in action2.implications):
        return True
    if any(elem in action1.effect for elem in action2.preconditions):
        return True
    if any(elem in action1.preconditions for elem in action2.effect):
        return True
    
    return False

In [67]:
def build_plan_graph(state, goal):
    grasp = GraspKey()
    lock = LockDoor()
    pk = PutKeyIntoBox()
    mv1 = Move('R2', 'R1')
    mv2 = Move('R1', 'R2')
    
    graph_levels = [state]
    
    i = 0
    while(True):
        # print(f'LEVEL {i+1}')
        new_level = [*graph_levels[i]]
        new_literals = grasp.action(graph_levels[i])
        for lit in new_literals:
            new_level.append(lit)
        new_literals = lock.action(graph_levels[i])
        # print(len(new_literals))
        for lit in new_literals:
            new_level.append(lit)
        new_literals = pk.action(graph_levels[i])
        for lit in new_literals:
            new_level.append(lit)
        new_literals = mv1.action(graph_levels[i])
        for lit in new_literals:
            new_level.append(lit)
        new_literals = mv2.action(graph_levels[i])
        for lit in new_literals:
            new_level.append(lit)
    
        graph_levels.append(list(set(new_level)))

        if all(elem in new_level for elem in goal):
            break
            
        if len(list(set(new_level))) == len(graph_levels[i]):
            return []
            break
        
        i += 1
            
    return graph_levels
        
        

In [68]:
def graph_heuristic(state, goal):
    graph = build_plan_graph(state, goal)
    if len(graph) > 0:
        return len(graph) - 1
    else:
        return float('inf')

In [69]:
grasp = GraspKey()
lock = LockDoor()
pk = PutKeyIntoBox()
mv1 = Move('R2', 'R1')
mv2 = Move('R1', 'R2')

In [70]:
ac = [grasp, lock, pk, mv1, mv2]

In [71]:
def get_possible_actions(action_list):
    not_mutex = []
    for i in range(len(action_list)):
        for j in range(i+1, len(action_list)):
            if not mutex(action_list[i], action_list[j]):
                # print('alg0')
                not_mutex.append([action_list[i], action_list[j]])
                
    for action in action_list:
        not_mutex.append([action])
    return not_mutex

In [72]:
actions = get_possible_actions(ac)

In [73]:
actions

[[<__main__.GraspKey at 0x1327e90b730>],

In [74]:
class Node:
    def __init__(self, state, cost, heuristic, plan):
        self.state = state
        self.cost = cost
        self.heuristic = heuristic
        self.function = cost + heuristic
        self.plan = plan

In [75]:
node = Node(initial_state, 0, float('inf'), [])

In [76]:
def A_star(initial_node, goal, actions):
    frontier = [initial_node]
    while len(frontier) > 0:
        node = frontier.pop(0)
        # print('------------------------------------------')
        # print(' ^ '.join([str(x) for x in node.state]))
        # print('------------------------------------------')
        if all(elem in node.state for elem in goal):
            return node.plan
            break
        else:
            for action in actions:
                for sub_action in action:
                    # print(sub_action)
                    new_state = sub_action.action(node.state)
                    # print(' ^ '.join([str(x) for x in new_state]))
                   
                if all(elem in node.state for elem in new_state) and all(elem in new_state for elem in node.state) :
                    pass
                else:
                    h = graph_heuristic(new_state, goal)
                    c = node.cost + 1
                    # print(node.plan)
                    # print(node.plan.append(action))
                    new_node = Node(new_state, c, h, copy.deepcopy(node.plan))
                    new_node.plan.append(action)
                    # print(' ^ '.join([str(x) for x in new_state]))
                    # print(new_node.cost, new_node.heuristic)
                    # print(new_node.plan)
                    # node.plan.append(str(action))
                    # new_node.plan = node.plan
                    
                    frontier.append(new_node)
            
            frontier = sorted(frontier, key = lambda x: x.function)
    return 'NO HAY RESPUESTA'

In [77]:
node = Node(initial_state, 0, float('inf'), [])
goal = [Locked('Door'), In('Key', 'Box')]
plan = A_star(node, goal, actions)

In [78]:
plan = ' -> '.join([str(item) for sublist in plan for item in sublist])

In [79]:
plan

'Grasp Key -> Move from R2 to R1 -> Lock Door -> Put the key into the box'